<a href="https://colab.research.google.com/github/maheshmistry/ProgrammT/blob/main/Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rdflib
!pip install faker
!git clone https://github.com/maheshmistry/ProgrammT.git

     |████████████████████████████████| 235kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 1.1MB 9.7MB/s 
Cloning into 'ProgrammT'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [3]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from faker import Faker
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random



dbo = Namespace("http://dbpedia.org/ontology/")
TOnto = Namespace("http://visualdataweb.org/MCompanyTest/")

fake = Faker()
g = Graph()

g.bind("TOnto", 'http://visualdataweb.org/MCompanyTest/')
g.bind("dbo", 'http://dbpedia.org/ontology/')

g.parse("/content/ProgrammT/NewOntology.rdf")

<Graph identifier=N9fd5360c24a3491d9750eff5bc8d2d5f (<class 'rdflib.graph.Graph'>)>

In [4]:
qres = g.query(
    """SELECT ?s ?p ?o
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s ?p ?o.
           filter (datatype(?o) != '')
       }""")

for row in qres:
    print("%s -- %s -- %s" % row)

http://visualdataweb.org/MCompanyTest/Google -- http://dbpedia.org/ontology/numberOfEmployees -- 40
http://visualdataweb.org/MCompanyTest/Google -- http://visualdataweb.org/MCompanyTest/avgSalary -- 80000
http://visualdataweb.org/MCompanyTest/Google -- http://dbpedia.org/ontology/industry -- IT
http://visualdataweb.org/MCompanyTest/Google -- http://visualdataweb.org/MCompanyTest/genderRatio -- 64.5
http://visualdataweb.org/MCompanyTest/Facebook -- http://dbpedia.org/ontology/industry -- Software
http://visualdataweb.org/MCompanyTest/Facebook -- http://dbpedia.org/ontology/numberOfEmployees -- 20
http://visualdataweb.org/MCompanyTest/Facebook -- http://visualdataweb.org/MCompanyTest/genderRatio -- 70.4
http://visualdataweb.org/MCompanyTest/Facebook -- http://visualdataweb.org/MCompanyTest/avgSalary -- 70000


In [8]:
enumb = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s dbo:numberOfEmployees ?o.
       }""")

avgSal = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE {
           ?s rdf:type TOnto:Company.
           ?s TOnto:avgSalary ?o.
       }""")
avgGenRat = g.query(
    """SELECT (AVG(?o) AS ?avg)
       WHERE{
          ?s rdf:type TOnto:Company.
          ?s TOnto:genderRatio ?o.
       }"""
)

for row in enumb:
  emp = row.avg
for row in avgSal:
  avgSale = row.avg
for row in avgGenRat:
  avgGenRatio = row.avg

emp = int(emp)
avgSale = float(avgSale)


In [14]:
synthGender = np.random.choice(["Male", "Female"], emp, p=[(avgGenRatio.value/100), ((100-avgGenRatio.value)/100)])

emp_roles = ['Engineer','Senior Engineer','Manager']
company = pd.DataFrame([
          {
              "FirstName": fake.first_name_male() if synthGender[n]=='Male' else fake.first_name_female(),
              "LastName": fake.last_name(),
              "Gender" : synthGender[n],
              "Salary" : random.randint(int(avgSale - (avgSale * .20)), int(avgSale + (avgSale * .20))),
              "Roles" : "CEO" if n == 0 else np.random.choice(emp_roles, p=[0.6, 0.3, 0.1])
          }for n in range(emp)
])



company

,FirstName,LastName,Gender,Salary,Roles
0,Kristopher,Hughes,Male,69928,CEO
1,Mitchell,Reyes,Male,78592,Engineer
2,Kathryn,Durham,Female,67030,Manager
3,Craig,Zavala,Male,81626,Engineer
4,Michael,Johnson,Male,74272,Engineer
5,Kevin,Diaz,Male,81729,Senior Engineer
6,Ryan,Brooks,Male,71390,Manager
7,Michelle,Clark,Female,84513,Engineer
8,Sharon,Baker,Female,62494,Senior Engineer
9,Lauren,Bonilla,Female,73336,Senior Engineer


In [15]:
genDist = (company['Gender'].value_counts(normalize=True) * 100)

print("New Company stats:"
      "\n\nNo. of Employees:" , emp ,
      "\n\nGender Ratio(Male Percent):\n", genDist,
      "\n\nAverage Salary:", sum(company["Salary"])/emp,
      "\n\nRole Distribution:\n", company["Roles"].value_counts())

New Company stats:

No. of Employees: 30 

Gender Ratio(Male Percent):
 Male      60.0
Female    40.0
Name: Gender, dtype: float64 

Average Salary: 77386.6 

Role Distribution:
 Engineer           14
Senior Engineer    11
Manager             4
CEO                 1
Name: Roles, dtype: int64
